In [11]:
# 使用 floweaver 和 holoviews 创建四列连续桑基图
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import matplotlib

# 设置全局字体为Arial
plt.rcParams.update({
    'font.family': 'Arial',  # 设置字体为Arial
    'font.size': 16,  # 基础字体大小
})

# =============================================================================
# 数据预处理部分 - 三个情景的四列桑基图数据准备
# =============================================================================

# 1. 读取岛屿可行性分析数据
data_path = Path("../result/island_viability_summary_electric.csv")
df = pd.read_csv(data_path)  # 加载原始数据
print(f"原始数据形状: {df.shape}")  # 显示数据维度信息

# 2. 加载IPCC区域地理数据用于区域分配
try:
    ipcc_regions = gpd.read_file("IPCC-WGI-reference-regions-v4.geojson")  # 加载IPCC区域边界数据
    ipcc_regions.sindex  # 创建空间索引以加速地理查询
    print("IPCC区域地理数据加载成功")
except Exception as e:
    print(f"无法加载IPCC区域数据: {e}")
    ipcc_regions = None

# 3. 定义IPCC区域分配函数（高效版本）
def assign_ipcc_region_optimized(lat, lon, ipcc_regions_gdf):
    """使用空间索引高效地将坐标分配到IPCC区域"""
    point = Point(lon, lat)  # 创建点几何对象
    possible_matches_index = list(ipcc_regions_gdf.sindex.intersection(point.bounds))  # 空间索引快速筛选
    possible_matches = ipcc_regions_gdf.iloc[possible_matches_index]  # 获取可能匹配的区域
    precise_matches = possible_matches[possible_matches.contains(point)]  # 精确几何匹配
    if not precise_matches.empty:
        return precise_matches.iloc[0]['Acronym']  # 返回区域简称
    return 'Unknown'  # 未匹配到区域时返回Unknown

# 4. 定义变化计算函数（与上面cell保持一致）
def calculate_position_change(df_base, df_compare):
    """计算两个情景之间的位置变化 - 修正版，包含IPCC区域信息"""
    # 修正：合并时包含ipcc_region信息
    merged = pd.merge(df_base[['island_id', 'tariff_breakeven', 'ipcc_region']],
                      df_compare[['island_id', 'tariff_breakeven']],
                      on='island_id', suffixes=('_base', '_compare'))
    merged['position_change'] = abs(merged['tariff_breakeven_compare'] - merged['tariff_breakeven_base'])
    # 重命名ipcc_region列以保持一致性
    merged = merged.rename(columns={'ipcc_region': 'ipcc_region_base'})
    return merged

# 5. 为整个数据集分配IPCC区域（与上面cell保持一致）
if ipcc_regions is not None:
    print("正在为岛屿分配IPCC区域...")
    df['ipcc_region'] = df.apply(
        lambda row: assign_ipcc_region_optimized(row['lat'], row['lon'], ipcc_regions), axis=1
    )  # 应用区域分配函数
    print(f"区域分配完成，涉及区域: {df['ipcc_region'].nunique()}")
else:
    df['ipcc_region'] = 'Unknown'  # 如果没有区域数据，设为Unknown

# =============================================================================
# NEW: 先筛选在2020年scenario有显著变化的岛屿
# =============================================================================

# print("\n=== 筛选在2020年scenario有显著变化的岛屿 ===")

# # 获取Ideal和2020年scenario的数据
# df_ideal = df[df['scenario'] == 'output_0'].copy()  # Ideal情景数据
# df_2020 = df[df['scenario'] == 'output_2020'].copy()  # 2020年情景数据

# # 计算变化
# change_data = calculate_position_change(df_ideal, df_2020)
# significant_threshold = 0.05  # 显著变化阈值
# significant_changes = change_data[change_data['position_change'] > significant_threshold]

# print(f"原始岛屿总数: {len(df_ideal)}")
# print(f"在2020年scenario有显著变化的岛屿数: {len(significant_changes)}")
# print(f"显著变化阈值: {significant_threshold}")

# # 获取有显著变化的岛屿ID列表
# significant_island_ids = significant_changes['island_id'].unique()
# print(f"有显著变化的唯一岛屿数: {len(significant_island_ids)}")

# # 从原始数据中筛选出这些有显著变化的岛屿
# df_filtered = df[df['island_id'].isin(significant_island_ids)].copy()
# print(f"筛选后的数据形状: {df_filtered.shape}")
df_filtered = df
# =============================================================================
# 6. 在有显著变化的岛屿中，再筛选有效区域（岛屿数量阈值）
# =============================================================================

print("\n=== 在有显著变化的岛屿中应用区域数量阈值 ===")

MIN_ISLANDS_PER_REGION = 10  # 每个区域最少岛屿数量阈值

# 统计有显著变化的岛屿在各区域的分布
region_counts_filtered = df_filtered['ipcc_region'].value_counts()  # 统计各区域岛屿数量
valid_regions = region_counts_filtered[region_counts_filtered > MIN_ISLANDS_PER_REGION].index.tolist()  # 筛选有效区域

print(f"有显著变化岛屿的区域分布:")
print(region_counts_filtered.head(10))
print(f"满足数量阈值的有效区域 (岛屿数量>{MIN_ISLANDS_PER_REGION}): {len(valid_regions)} 个")
print(f"有效区域列表: {valid_regions}")

# 最终筛选：既有显著变化又在有效区域的岛屿
df_final = df_filtered[df_filtered['ipcc_region'].isin(valid_regions)].copy()
final_island_ids = df_final['island_id'].unique()

print(f"最终用于桑基图的岛屿数: {len(final_island_ids)}")
print(f"最终数据形状: {df_final.shape}")

# =============================================================================
# 7. 定义象限分类函数
# =============================================================================

def classify_quadrant(row, median_breakeven, median_affordable):
    """根据电价中位数将岛屿分类到四个象限"""
    breakeven = row['tariff_breakeven']  # 盈亏平衡电价
    affordable = row['tariff_affordable']  # 可负担电价

    if affordable > median_affordable and breakeven <= median_breakeven:
        return 'High Affordable\\nLow Cost'  # 高可负担性、低成本
    elif affordable > median_affordable and breakeven > median_breakeven:
        return 'High Affordable\\nHigh Cost'  # 高可负担性、高成本
    elif affordable <= median_affordable and breakeven <= median_breakeven:
        return 'Low Affordable\\nLow Cost'  # 低可负担性、低成本
    else:
        return 'Low Affordable\\nHigh Cost'  # 低可负担性、高成本

# =============================================================================
# 8. 处理三个情景的数据并计算各自的中位数分类
# =============================================================================

scenarios = ['output_0', 'output_2020', 'output_2050']  # 三个主要情景
scenario_labels = ['Ideal', 'Baseline', 'Climate Stress']  # 情景标签

# 准备合并数据的列表
combined_data = []

# 为每个情景计算中位数并分类（只针对最终筛选的岛屿）
for scenario in scenarios:
    df_scenario = df_final[df_final['scenario'] == scenario].copy()  # 筛选情景数据

    # 计算该情景下的中位数阈值
    median_breakeven = df_scenario['tariff_breakeven'].median()  # 盈亏平衡电价中位数
    median_affordable = df_scenario['tariff_affordable'].median()  # 可负担电价中位数

    # 应用象限分类
    df_scenario['quadrant'] = df_scenario.apply(
        lambda row: classify_quadrant(row, median_breakeven, median_affordable), axis=1
    )  # 为每个岛屿分配象限

    print(f"情景 {scenario}: 有效岛屿数 {len(df_scenario)}, 中位数 - 盈亏平衡: {median_breakeven:.3f}, 可负担: {median_affordable:.3f}")

    combined_data.append(df_scenario)

# =============================================================================
# 9. 创建四列桑基图的数据结构
# =============================================================================

def prepare_four_column_sankey_data(df_list):
    """为四列桑基图准备数据"""

    # 找到所有情景中共同存在的岛屿
    common_islands = set(df_list[0]['island_id'])
    for df_scenario in df_list[1:]:
        common_islands = common_islands.intersection(set(df_scenario['island_id']))

    print(f"三个情景共同存在的岛屿数量: {len(common_islands)}")

    # 为共同岛屿创建桑基图数据
    sankey_data = []

    for island_id in common_islands:
        # 获取该岛屿在三个情景下的信息
        island_info = {}
        for i, df_scenario in enumerate(df_list):
            island_row = df_scenario[df_scenario['island_id'] == island_id].iloc[0]
            island_info[f'scenario_{i}'] = {
                'region': island_row['ipcc_region'],
                'quadrant': island_row['quadrant']
            }

        # 添加到桑基图数据
        sankey_data.append({
            'island_id': island_id,
            'region': island_info['scenario_0']['region'],  # 第一列：区域
            'ideal_class': island_info['scenario_0']['quadrant'],  # 第二列：Ideal分类
            'baseline_class': island_info['scenario_1']['quadrant'],  # 第三列：Baseline分类
            'climate_class': island_info['scenario_2']['quadrant']  # 第四列：Climate Stress分类
        })

    return pd.DataFrame(sankey_data)

# =============================================================================
# 10. 准备桑基图数据
# =============================================================================

sankey_df = prepare_four_column_sankey_data(combined_data)
print(f"\n=== 最终桑基图数据摘要 ===")
print(f"桑基图数据准备完成，包含 {len(sankey_df)} 个岛屿")
print(f"涉及区域: {sankey_df['region'].nunique()} 个")
print(f"区域列表: {sorted(sankey_df['region'].unique())}")

# 打印筛选过程摘要
print(f"\n=== 筛选过程摘要 ===")
print(f"1. 原始岛屿总数: {len(df_ideal)}")
print(f"2. 在2020年有显著变化的岛屿: {len(significant_island_ids)}")
print(f"3. 满足区域数量阈值的岛屿: {len(final_island_ids)}")
print(f"4. 最终桑基图岛屿数: {len(sankey_df)}")
print(f"5. 最终涉及区域数: {len(valid_regions)}")

# --- 可选：将数据保存为 CSV 文件，供 R 使用 ---
# output_path = "sankey_data_for_r.csv"
# sankey_df.to_csv(output_path, index=False)
# print(f"数据已成功导出到: {output_path}")

原始数据形状: (10980, 16)
IPCC区域地理数据加载成功
正在为岛屿分配IPCC区域...
区域分配完成，涉及区域: 43

=== 在有显著变化的岛屿中应用区域数量阈值 ===
有显著变化岛屿的区域分布:
ipcc_region
SEA    4188
EAS    1002
NEU     924
MED     690
EPO     528
SAS     528
SPO     504
CAR     366
ENA     270
WAF     192
Name: count, dtype: int64
满足数量阈值的有效区域 (岛屿数量>10): 39 个
有效区域列表: ['SEA', 'EAS', 'NEU', 'MED', 'EPO', 'SAS', 'SPO', 'CAR', 'ENA', 'WAF', 'NAU', 'NAO', 'SWS', 'NWN', 'WNA', 'ARP', 'SCA', 'NPO', 'NZ', 'BOB', 'SSA', 'RFE', 'MDG', 'NCA', 'SAH', 'SAU', 'WCA', 'WCE', 'SEAF', 'NEN', 'ESAF', 'EIO', 'NES', 'EAU', 'SIO', 'GIC', 'CAF', 'EAO', 'NSA']
最终用于桑基图的岛屿数: 1825
最终数据形状: (10956, 17)
情景 output_0: 有效岛屿数 1826, 中位数 - 盈亏平衡: 0.197, 可负担: 0.146
情景 output_2020: 有效岛屿数 1826, 中位数 - 盈亏平衡: 0.199, 可负担: 0.146
情景 output_2050: 有效岛屿数 1826, 中位数 - 盈亏平衡: 0.199, 可负担: 0.146
三个情景共同存在的岛屿数量: 1825

=== 最终桑基图数据摘要 ===
桑基图数据准备完成，包含 1825 个岛屿
涉及区域: 39 个
区域列表: ['ARP', 'BOB', 'CAF', 'CAR', 'EAO', 'EAS', 'EAU', 'EIO', 'ENA', 'EPO', 'ESAF', 'GIC', 'MDG', 'MED', 'NAO', 'NAU', 'NCA', 'NEN', 'NES',

In [12]:
sankey_df

,island_id,region,ideal_class,baseline_class,climate_class
0,65545,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
1,49163,NEU,High Affordable\nHigh Cost,High Affordable\nHigh Cost,High Affordable\nHigh Cost
2,40971,NEU,High Affordable\nHigh Cost,High Affordable\nHigh Cost,High Affordable\nHigh Cost
3,73748,SEA,High Affordable\nHigh Cost,High Affordable\nHigh Cost,High Affordable\nHigh Cost
4,65558,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
...,...,...,...,...,...
1820,40937,NEU,High Affordable\nHigh Cost,High Affordable\nHigh Cost,High Affordable\nHigh Cost
1821,40939,NEU,High Affordable\nHigh Cost,High Affordable\nHigh Cost,High Affordable\nHigh Cost
1822,73708,SEA,Low Affordable\nLow Cost,Low Affordable\nHigh Cost,Low Affordable\nHigh Cost
1823,65517,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost


In [13]:
# 1. 识别发生过分类变化的岛屿
changed_island_ids = []
for _, row in sankey_df.iterrows():
    # 如果三个情景的分类不完全相同，就认为发生了变化
    if not (row['ideal_class'] == row['baseline_class'] == row['climate_class']):      
        changed_island_ids.append(row['island_id'])

print(f"发生过分类变化的岛屿数量: {len(changed_island_ids)}")

# 2. 获取这些变化岛屿所在的区域
changed_regions = sankey_df[sankey_df['island_id'].isin(changed_island_ids)]['region'].unique()
print(f"有岛屿发生变化的区域: {sorted(changed_regions)}")

# 3. 筛选出这些区域内的所有岛屿（包括未变化的）
sankey_df_filtered = sankey_df[sankey_df['region'].isin(changed_regions)].copy()       

print(f"原始数据: {len(sankey_df)} 个岛屿")
print(f"筛选后数据: {len(sankey_df_filtered)} 个岛屿")

# 现在sankey_df_filtered就是你要的结果
sankey_df_filtered

发生过分类变化的岛屿数量: 162
有岛屿发生变化的区域: ['BOB', 'EAS', 'MDG', 'MED', 'NAU', 'NPO', 'NZ', 'SAS', 'SAU', 'SCA', 'SEA', 'SIO', 'SPO', 'SWS', 'WCE']
原始数据: 1825 个岛屿
筛选后数据: 1283 个岛屿


,island_id,region,ideal_class,baseline_class,climate_class
0,65545,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
3,73748,SEA,High Affordable\nHigh Cost,High Affordable\nHigh Cost,High Affordable\nHigh Cost
4,65558,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
7,73776,SEA,Low Affordable\nHigh Cost,Low Affordable\nHigh Cost,Low Affordable\nHigh Cost
9,73787,SEA,Low Affordable\nHigh Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
...,...,...,...,...,...
1818,65511,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
1819,73704,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost
1822,73708,SEA,Low Affordable\nLow Cost,Low Affordable\nHigh Cost,Low Affordable\nHigh Cost
1823,65517,SEA,Low Affordable\nLow Cost,Low Affordable\nLow Cost,Low Affordable\nLow Cost


In [14]:
sankey_df_filtered.to_csv('chongji_for_R.csv', index=False)

In [3]:
# 9. --- Nature Communications风格四层桑基图：Region -> Ideal -> Baseline -> Climate Stress ---

import plotly.graph_objects as go  # 导入plotly桑基图绘制库
import plotly.io as pio  # 导入plotly IO库用于格式设置
import matplotlib.cm as cm  # matplotlib颜色映射库
import matplotlib.colors as mcolors  # matplotlib颜色工具
import numpy as np  # 数值计算库

print("开始创建Nature Communications风格四层桑基图（仿照output.png样式）...")

# 设置plotly默认样式为静态图，符合期刊要求
pio.renderers.default = "browser"  # 设置输出渲染器为浏览器

# =============================================================================
# 1. 数据检查和预处理
# =============================================================================

# 检查桑基图数据是否存在
if 'sankey_df' not in locals():
    print("错误：sankey_df数据不存在，请先运行上一个cell生成数据")
else:
    print(f"桑基图数据检查通过，包含 {len(sankey_df)} 个岛屿")
    print(f"涉及区域: {sankey_df['region'].nunique()} 个")
    print(f"涉及分类组合: {len(sankey_df[['ideal_class', 'baseline_class', 'climate_class']].drop_duplicates())} 种")

# =============================================================================
# 2. Nature风格颜色映射 - 仿照output.png的简洁配色
# =============================================================================

# 定义四个主要类别的颜色（仿照output.png）
category_colors = {
    'High Affordable\\nLow Cost': 'rgba(255, 223, 0, 0.85)',      # 金黄色，对应output.png底部
    'High Affordable\\nHigh Cost': 'rgba(255, 127, 127, 0.85)',   # 粉红色，对应output.png中部偏下
    'Low Affordable\\nLow Cost': 'rgba(64, 224, 224, 0.85)',      # 青色，对应output.png主体
    'Low Affordable\\nHigh Cost': 'rgba(255, 140, 0, 0.85)'       # 橙色，对应output.png上部
}

print(f"使用Nature期刊风格的四色配色方案")

# 为区域生成基于类别主色调的变化色彩
def generate_region_colors_from_categories(regions, base_colors):
    """基于四个主要类别颜色生成区域颜色变化"""
    region_colors = {}
    base_color_list = list(base_colors.values())
    n_regions = len(regions)
    
    for i, region in enumerate(regions):
        # 循环使用四种基础颜色，并添加微调
        base_idx = i % 4
        base_color = base_color_list[base_idx]
        
        # 提取RGB值并进行微调
        rgb_match = base_color.replace('rgba(', '').replace(')', '').split(', ')
        r, g, b = int(rgb_match[0]), int(rgb_match[1]), int(rgb_match[2])
        
        # 为每个区域添加细微的色彩变化
        variation = (i // 4) * 15  # 每4个区域为一组，组间有色彩变化
        r = min(255, max(0, r + variation - 30))
        g = min(255, max(0, g + variation - 15))  
        b = min(255, max(0, b + variation - 10))
        
        region_colors[region] = f'rgba({r}, {g}, {b}, 0.75)'
    
    return region_colors

# 获取所有唯一区域并生成颜色
unique_regions = sorted(sankey_df['region'].unique())
region_colors = generate_region_colors_from_categories(unique_regions, category_colors)

print(f"为 {len(unique_regions)} 个区域生成了基于类别的颜色变化")

# =============================================================================
# 3. 构建四层桑基图的节点和链接数据结构
# =============================================================================

# 定义四个层级的所有可能节点
all_regions = sorted(sankey_df['region'].unique())  # 第1层：区域
all_ideal_classes = sorted(sankey_df['ideal_class'].unique())  # 第2层：Ideal分类
all_baseline_classes = sorted(sankey_df['baseline_class'].unique())  # 第3层：Baseline分类
all_climate_classes = sorted(sankey_df['climate_class'].unique())  # 第4层：Climate Stress分类

# 创建节点标签列表（按层级顺序）- 仿照output.png的简洁标签
node_labels = []
node_colors = []  # 节点颜色列表

# 第1层：区域节点 - 使用区域代码（仿照output.png左侧标签）
for region in all_regions:
    node_labels.append(region)  # 简洁的区域标签
    node_colors.append(region_colors[region])  # 使用区域对应颜色

# 第2层：Ideal分类节点 - 使用类别颜色
for ideal_class in all_ideal_classes:
    node_labels.append("")  # 不显示中间节点标签，仿照output.png
    node_colors.append(category_colors[ideal_class])  # 使用类别对应颜色

# 第3层：Baseline分类节点 - 使用类别颜色
for baseline_class in all_baseline_classes:
    node_labels.append("")  # 不显示中间节点标签
    node_colors.append(category_colors[baseline_class])

# 第4层：Climate Stress分类节点 - 显示最终类别标签（仿照output.png右侧）
for climate_class in all_climate_classes:
    # 简化标签，仿照output.png的样式
    if climate_class == 'High Affordable\\nLow Cost':
        display_label = "High Affordable\nLow Cost"
    elif climate_class == 'High Affordable\\nHigh Cost':
        display_label = "High Affordable\nHigh Cost"  
    elif climate_class == 'Low Affordable\\nLow Cost':
        display_label = "Low Affordable\nLow Cost"
    else:  # 'Low Affordable\\nHigh Cost'
        display_label = "Low Affordable\nHigh Cost"
    
    node_labels.append(display_label)
    node_colors.append(category_colors[climate_class])

print(f"创建了 {len(node_labels)} 个节点，仿照output.png的标签样式")

# 创建节点索引映射字典
node_indices = {label: i for i, label in enumerate(node_labels)}

# 为空标签节点创建特殊映射
region_start_idx = 0
ideal_start_idx = len(all_regions)
baseline_start_idx = len(all_regions) + len(all_ideal_classes)
climate_start_idx = len(all_regions) + len(all_ideal_classes) + len(all_baseline_classes)

def get_node_index(layer, item):
    """获取节点索引"""
    if layer == 'region':
        return all_regions.index(item)
    elif layer == 'ideal':
        return ideal_start_idx + all_ideal_classes.index(item)
    elif layer == 'baseline':
        return baseline_start_idx + all_baseline_classes.index(item)
    else:  # climate
        return climate_start_idx + all_climate_classes.index(item)

# =============================================================================
# 4. 构建链接数据 - 保持region颜色的连续性
# =============================================================================

# 初始化链接列表
source_indices = []
target_indices = []
link_values = []
link_colors = []

print("开始构建链接关系...")

# --- 第1层到第2层的链接：Region -> Ideal ---
region_to_ideal = sankey_df.groupby(['region', 'ideal_class']).size().reset_index(name='count')
for _, row in region_to_ideal.iterrows():
    source_idx = get_node_index('region', row['region'])
    target_idx = get_node_index('ideal', row['ideal_class'])
    
    source_indices.append(source_idx)
    target_indices.append(target_idx)
    link_values.append(row['count'])
    # 使用稍微透明的区域颜色，仿照output.png的流动效果
    link_colors.append(region_colors[row['region']].replace('0.75)', '0.6)'))

print(f"第1层到第2层链接: {len(region_to_ideal)} 条")

# --- 第2层到第3层的链接：Ideal -> Baseline ---
ideal_to_baseline = sankey_df.groupby(['region', 'ideal_class', 'baseline_class']).size().reset_index(name='count')
for _, row in ideal_to_baseline.iterrows():
    source_idx = get_node_index('ideal', row['ideal_class'])
    target_idx = get_node_index('baseline', row['baseline_class'])
    
    source_indices.append(source_idx)
    target_indices.append(target_idx)
    link_values.append(row['count'])
    # 使用区域颜色保持连续性
    link_colors.append(region_colors[row['region']].replace('0.75)', '0.6)'))

print(f"第2层到第3层链接: {len(ideal_to_baseline)} 条")

# --- 第3层到第4层的链接：Baseline -> Climate Stress ---
baseline_to_climate = sankey_df.groupby(['region', 'baseline_class', 'climate_class']).size().reset_index(name='count')
for _, row in baseline_to_climate.iterrows():
    source_idx = get_node_index('baseline', row['baseline_class'])
    target_idx = get_node_index('climate', row['climate_class'])
    
    source_indices.append(source_idx)
    target_indices.append(target_idx)
    link_values.append(row['count'])
    # 使用区域颜色保持连续性
    link_colors.append(region_colors[row['region']].replace('0.75)', '0.6)'))

print(f"第3层到第4层链接: {len(baseline_to_climate)} 条")
print(f"总链接数: {len(source_indices)}")

# =============================================================================
# 5. 创建Nature风格的桑基图 - 仿照output.png
# =============================================================================

# 创建桑基图对象
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=20,  # 较小的节点间距，仿照output.png的紧凑感
        thickness=25,  # 较细的节点厚度，更加优雅
        line=dict(color="rgba(200, 200, 200, 0.5)", width=0.8),  # 很淡的边框
        label=node_labels,
        color=node_colors,
        # 仿照output.png的四列布局，稍微调整间距
        x=[0.05 if i < len(all_regions) else
           0.35 if i < len(all_regions) + len(all_ideal_classes) else
           0.65 if i < len(all_regions) + len(all_ideal_classes) + len(all_baseline_classes) else
           0.95
           for i in range(len(node_labels))],
        y=[i/(max(len(all_regions)-1, 1)) if i < len(all_regions) else
           (i-len(all_regions))/(max(len(all_ideal_classes)-1, 1)) if i < len(all_regions) + len(all_ideal_classes) else
           (i-len(all_regions)-len(all_ideal_classes))/(max(len(all_baseline_classes)-1, 1)) if i < len(all_regions) + len(all_ideal_classes) + len(all_baseline_classes) else
           (i-len(all_regions)-len(all_ideal_classes)-len(all_baseline_classes))/(max(len(all_climate_classes)-1, 1))
           for i in range(len(node_labels))]
    ),
    link=dict(
        source=source_indices,
        target=target_indices,
        value=link_values,
        color=link_colors,
        line=dict(color="rgba(255, 255, 255, 0.3)", width=0.3)  # 很淡的链接边框，仿照output.png
    )
)])

# =============================================================================
# 6. 应用output.png的简洁样式设置
# =============================================================================

fig.update_layout(
    font=dict(
        family="Arial",  # Nature期刊标准字体
        size=18,         # 适中的字体大小，仿照output.png
        color="rgb(60, 60, 60)"  # 深灰色文字，不是纯黑
    ),
    plot_bgcolor='rgb(255, 255, 255)',    # 纯白背景，仿照output.png
    paper_bgcolor='rgb(255, 255, 255)',   # 纯白纸张背景
    width=1200,              # 稍微窄一些，仿照output.png的比例
    height=800,              # 适中的高度
    margin=dict(
        l=50,   # 较小的边距，更加紧凑
        r=150,  # 右边距大一些，为标签留空间
        t=30,   # 较小的上边距
        b=30    # 较小的下边距
    ),
    showlegend=False  # 不显示图例，符合output.png的简洁风格
)

# 显示桑基图
fig.show()

# =============================================================================
# 7. 打印样式优化摘要
# =============================================================================

print(f"\n=== Nature风格桑基图（仿照output.png）样式摘要 ===")
print(f"✓ 采用四色主配色方案：青色、橙色、粉红色、金黄色")
print(f"✓ 简洁的节点设计：厚度25，间距20")
print(f"✓ 优雅的流动效果：链接透明度0.6，边框极淡")
print(f"✓ 紧凑的布局：1200×800尺寸，类似output.png比例")
print(f"✓ 清晰的标签系统：左侧区域标签，右侧类别标签")

print(f"\n数据统计:")
print(f"总岛屿数: {len(sankey_df)}")
print(f"涉及区域数: {len(all_regions)}")
print(f"总节点数: {len(node_labels)}")
print(f"总链接数: {len(source_indices)}")

print(f"\n=== 仿照output.png样式的Nature风格桑基图完成 ===")
print("✓ 简洁优雅的颜色方案")
print("✓ 清晰的视觉层次")
print("✓ 期刊级别的图形质量")
print("✓ 保持完整的数据流追踪")

开始创建Nature Communications风格四层桑基图（仿照output.png样式）...
桑基图数据检查通过，包含 167 个岛屿
涉及区域: 13 个
涉及分类组合: 9 种
使用Nature期刊风格的四色配色方案
为 13 个区域生成了基于类别的颜色变化
创建了 25 个节点，仿照output.png的标签样式
开始构建链接关系...
第1层到第2层链接: 18 条
第2层到第3层链接: 22 条
第3层到第4层链接: 23 条
总链接数: 63

=== Nature风格桑基图（仿照output.png）样式摘要 ===
✓ 采用四色主配色方案：青色、橙色、粉红色、金黄色
✓ 简洁的节点设计：厚度25，间距20
✓ 优雅的流动效果：链接透明度0.6，边框极淡
✓ 紧凑的布局：1200×800尺寸，类似output.png比例
✓ 清晰的标签系统：左侧区域标签，右侧类别标签

数据统计:
总岛屿数: 167
涉及区域数: 13
总节点数: 25
总链接数: 63

=== 仿照output.png样式的Nature风格桑基图完成 ===
✓ 简洁优雅的颜色方案
✓ 清晰的视觉层次
✓ 期刊级别的图形质量
✓ 保持完整的数据流追踪
